In [1]:
import sys

import sklearn
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import keras

%matplotlib inline

/home/thalvadzhiev_gmail_com/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
import pickle

In [3]:
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Convolution2D, MaxPooling2D
from keras.layers import ZeroPadding2D
from keras.callbacks import ModelCheckpoint, TensorBoard
from keras.utils import np_utils
from keras.layers.normalization import BatchNormalization

In [4]:
with open("faces_dataset_conv.pickle", "rb") as f:
    data = pickle.load(f)

In [36]:
train_ds = data["train_dataset"]
train_labels = data["train_labels"]
pub_test_ds = data["public_test"]
pub_test_labels = data["public_test_labels"]
priv_test_ds = data["private_test"]
priv_test_labels = data["private_test_labels"]

In [24]:
train_labels[:10]

array([0, 0, 2, 4, 6, 2, 4, 3, 3, 2], dtype=int64)

In [6]:
train_ds = train_ds.reshape(train_ds.shape[0], 1, 48, 48)
pub_test_ds = pub_test_ds.reshape(pub_test_ds.shape[0], 1, 48, 48)
priv_test_ds = priv_test_ds.reshape(priv_test_ds.shape[0], 1, 48, 48)

In [18]:
train_bigger = np.vstack([train_ds, pub_test_ds])

In [19]:
train_bigger.shape

(32298, 1, 48, 48)

In [7]:
train_labels = np_utils.to_categorical(train_labels, 7)
pub_test_labels = np_utils.to_categorical(pub_test_labels, 7)
priv_test_labels = np_utils.to_categorical(priv_test_labels, 7)

In [20]:
train_bigger_labels = np.concatenate([train_labels, pub_test_labels])

In [37]:
train_ds_inv = train_ds.reshape(train_ds.shape[0], 48, 48, 1)
pub_test_ds_inv = pub_test_ds.reshape(pub_test_ds.shape[0], 48, 48, 1)
priv_test_ds_inv = priv_test_ds.reshape(priv_test_ds.shape[0],48, 48, 1)
train_bigger_inv = np.vstack([train_ds_inv, pub_test_ds_inv])

train_labels = np_utils.to_categorical(train_labels, 7)
pub_test_labels = np_utils.to_categorical(pub_test_labels, 7)
priv_test_labels = np_utils.to_categorical(priv_test_labels, 7)
train_bigger_labels = np.concatenate([train_labels, pub_test_labels])

In [21]:
train_bigger_labels.shape

(32298, 7)

In [27]:
train_labels[:10]

array([[1., 0., 0., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0., 0., 0.],
       [0., 0., 1., 0., 0., 0., 0.],
       [0., 0., 0., 0., 1., 0., 0.],
       [0., 0., 0., 0., 0., 0., 1.],
       [0., 0., 1., 0., 0., 0., 0.],
       [0., 0., 0., 0., 1., 0., 0.],
       [0., 0., 0., 1., 0., 0., 0.],
       [0., 0., 0., 1., 0., 0., 0.],
       [0., 0., 1., 0., 0., 0., 0.]])

In [8]:
def get_221_model(weights=None):
    model = Sequential()
    model.add(ZeroPadding2D(padding=(3, 3), input_shape=(1, 48, 48)))
    model.add(Convolution2D(filters=128, kernel_size=4))
    model.add(BatchNormalization())
    model.add(Activation("relu"))
    model.add(Dropout(0.2))
    
    model.add(ZeroPadding2D(padding=(3, 3)))
    model.add(Convolution2D(filters=128, kernel_size=4))
    model.add(BatchNormalization())
    model.add(Activation("relu"))
    
    model.add(Convolution2D(filters=128, kernel_size=4, padding="same"))
    model.add(BatchNormalization())
    model.add(Activation("relu"))
    
    model.add(MaxPooling2D(pool_size=2, strides=2))
    model.add(Dropout(0.2))
    
    model.add(ZeroPadding2D(padding=(3, 3)))
    model.add(Convolution2D(filters=128, kernel_size=4))
    model.add(BatchNormalization())
    model.add(Activation("relu"))
    
    model.add(Convolution2D(filters=128, kernel_size=4, padding="same"))
    model.add(BatchNormalization())
    model.add(Activation("relu"))
    
    model.add(MaxPooling2D(pool_size=2, strides=2))
    model.add(Dropout(0.2))
    
    model.add(ZeroPadding2D(padding=(3, 3)))
    model.add(Convolution2D(filters=128, kernel_size=4))
    model.add(BatchNormalization())
    model.add(Activation("relu"))
    
    model.add(Convolution2D(filters=128, kernel_size=4, padding="same"))
    model.add(BatchNormalization())
    model.add(Activation("relu"))
    
    model.add(MaxPooling2D(pool_size=2, strides=2))
    model.add(Dropout(0.2))
    
    model.add(Flatten())
    ##fully connected
    model.add(Dense(units=1024))
    model.add(BatchNormalization())
    model.add(Activation("relu"))
    model.add(Dropout(0.2))
    
    model.add(Dense(units=1024))
    model.add(BatchNormalization())
    model.add(Activation("relu"))
    model.add(Dropout(0.2))
    
    model.add(Dense(units=7, activation="softmax"))
    
    if weights:
        model.load_weights(weights)
    
    return model


In [9]:
model = get_221_model("weights-improvement-04-0.41.hdf5")

In [10]:
tensorboard = TensorBoard(log_dir='./Graph', histogram_freq=1,write_graph=True, write_images=True)

In [11]:
filepath = "weights-improvement-{epoch:02d}-{val_acc:.2f}.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='val_acc', verbose=1, save_best_only=True, mode='max')
callback_list = [checkpoint, tensorboard]

In [12]:
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [22]:
model.fit(train_bigger, train_bigger_labels, 
          batch_size=128, validation_split=0.2, epochs=15, verbose=1, callbacks=callback_list)
model.save_weights("best_weights.hdf5")

Train on 25838 samples, validate on 6460 samples
Epoch 1/15
25838/25838 [==============================] - 50s 2ms/step - loss: 0.9733 - acc: 0.6411 - val_loss: 1.5069 - val_acc: 0.4772

Epoch 00001: val_acc did not improve from 0.49425
Epoch 2/15
25838/25838 [==============================] - 49s 2ms/step - loss: 0.9072 - acc: 0.6631 - val_loss: 1.4625 - val_acc: 0.4954

Epoch 00002: val_acc improved from 0.49425 to 0.49536, saving model to weights-improvement-02-0.50.hdf5
Epoch 3/15
25838/25838 [==============================] - 49s 2ms/step - loss: 0.8475 - acc: 0.6845 - val_loss: 1.4956 - val_acc: 0.4967

Epoch 00003: val_acc improved from 0.49536 to 0.49675, saving model to weights-improvement-03-0.50.hdf5
Epoch 4/15
25838/25838 [==============================] - 49s 2ms/step - loss: 0.7710 - acc: 0.7145 - val_loss: 1.5504 - val_acc: 0.5012

Epoch 00004: val_acc improved from 0.49675 to 0.50124, saving model to weights-improvement-04-0.50.hdf5
Epoch 5/15
25838/25838 [=============

In [25]:
score = model.evaluate(priv_test_ds, priv_test_labels, verbose=0)

In [26]:
score

[2.2812914079728315, 0.5018110894482593]

In [41]:
import h5py

In [28]:
def get_221_model2(weights=None):
    model = Sequential()
    model.add(ZeroPadding2D(padding=(3, 3), input_shape=(48, 48, 1)))
    model.add(Convolution2D(filters=128, kernel_size=4, data_format="channels_last"))
    model.add(BatchNormalization())
    model.add(Activation("relu"))
    model.add(Dropout(0.2))
    
    model.add(ZeroPadding2D(padding=(3, 3)))
    model.add(Convolution2D(filters=128, kernel_size=4, data_format="channels_last"))
    model.add(BatchNormalization())
    model.add(Activation("relu"))
    
    model.add(Convolution2D(filters=128, kernel_size=4, padding="same", data_format="channels_last"))
    model.add(BatchNormalization())
    model.add(Activation("relu"))
    
    model.add(MaxPooling2D(pool_size=2, strides=2))
    model.add(Dropout(0.2))
    
    model.add(ZeroPadding2D(padding=(3, 3)))
    model.add(Convolution2D(filters=128, kernel_size=4, data_format="channels_last"))
    model.add(BatchNormalization())
    model.add(Activation("relu"))
    
    model.add(Convolution2D(filters=128, kernel_size=4, padding="same", data_format="channels_last"))
    model.add(BatchNormalization())
    model.add(Activation("relu"))
    
    model.add(MaxPooling2D(pool_size=2, strides=2))
    model.add(Dropout(0.2))
    
    model.add(ZeroPadding2D(padding=(3, 3)))
    model.add(Convolution2D(filters=128, kernel_size=4, data_format="channels_last"))
    model.add(BatchNormalization())
    model.add(Activation("relu"))
    
    model.add(Convolution2D(filters=128, kernel_size=4, padding="same", data_format="channels_last"))
    model.add(BatchNormalization())
    model.add(Activation("relu"))
    
    model.add(MaxPooling2D(pool_size=2, strides=2))
    model.add(Dropout(0.2))
    
    model.add(Flatten())
    ##fully connected
    model.add(Dense(units=1024))
    model.add(BatchNormalization())
    model.add(Activation("relu"))
    model.add(Dropout(0.2))
    
    model.add(Dense(units=1024))
    model.add(BatchNormalization())
    model.add(Activation("relu"))
    model.add(Dropout(0.2))
    
    model.add(Dense(units=7, activation="softmax"))
    
    if weights:
        model.load_weights(weights)
    
    return model

In [29]:
model2 = get_221_model2()

In [30]:
filepath = "weights-improvement-model2-{epoch:02d}-{val_acc:.2f}.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='val_acc', verbose=1, save_best_only=True, mode='max')
callback_list = [checkpoint]

In [34]:
model2.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [40]:
model2.fit(train_bigger_inv, train_bigger_labels, 
          batch_size=128, validation_split=0.1, epochs=10, verbose=1, callbacks=callback_list)
model2.save_weights("best_weights_model2.hdf5")

Train on 29068 samples, validate on 3230 samples
Epoch 1/10
29068/29068 [==============================] - 182s 6ms/step - loss: 0.6444 - acc: 0.7703 - val_loss: 1.1706 - val_acc: 0.5873

Epoch 00001: val_acc improved from 0.57554 to 0.58731, saving model to weights-improvement-model2-01-0.59.hdf5
Epoch 2/10
29068/29068 [==============================] - 182s 6ms/step - loss: 0.5478 - acc: 0.8020 - val_loss: 1.3370 - val_acc: 0.5969

Epoch 00002: val_acc improved from 0.58731 to 0.59690, saving model to weights-improvement-model2-02-0.60.hdf5
Epoch 3/10
29068/29068 [==============================] - 182s 6ms/step - loss: 0.4487 - acc: 0.8355 - val_loss: 1.5784 - val_acc: 0.5604

Epoch 00003: val_acc did not improve from 0.59690
Epoch 4/10
29068/29068 [==============================] - 182s 6ms/step - loss: 0.3885 - acc: 0.8599 - val_loss: 1.6422 - val_acc: 0.5712

Epoch 00004: val_acc did not improve from 0.59690
Epoch 5/10
29068/29068 [==============================] - 182s 6ms/step -

In [42]:
score = model2.evaluate(priv_test_ds_inv, priv_test_labels, verbose=0)

In [43]:
score

[1.7575682035957947, 0.611869601568627]

In [39]:
!cat ~/.keras/keras.json

{
    "floatx": "float32",
    "epsilon": 1e-07,
    "backend": "tensorflow",
    "image_data_format": "channels_last"
}